# Lab 2: Network Intrusion Detection

In [17]:
# Imports
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics as mt
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
import time

from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

from sklearn.naive_bayes import GaussianNB

from sklearn.decomposition import RandomizedPCA 
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.pipeline import Pipeline
from sklearn import metrics as mt

import seaborn as sns
import pandas as pd
import numpy as np


%matplotlib inline

## Data Preparation:

* Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

In [13]:
# Load UNSW_NB15 into a Pandas dataframe
df = pd.read_csv('UNSW_NB15_training_set.csv', encoding='utf-8-sig')

# Lets remove attributes that are not useful to us during this first analysis pass
non_useful_features_list = ['id', 'attack_cat']
# id: n internal variable to just ref an obseration. deemed not usefl
# attack_cat: first try and just predict the label. 
#             It will obviously 1:1 correlate with label
#             We can circle back and swap it out with label 
#             to see if we get any better accuracy on an 
#             on an attack type level
for feature in non_useful_features_list:
    if feature in df:
        df.drop(feature, axis=1, inplace=True)  # Lets drop id as it is an internal variable to just ref an obseratio
        
# Overwrite the existing dataframe with the new dataframe that does not contain the 
# four unwanted records and confirm we have 4 less records (shold have 82328 observations)
if "is_ftp_login" in df:
    df = df[df.is_ftp_login != 2]
    if len(df) == 82328:
        print ("duplicate record deleted successfully: " + str(len(df)) + " observations remaining" )
        
# Check to see if non useful features still exist in dataframe, if so, we did something wrong
for feature in non_useful_features_list:
    if feature in df:
        print ("[" + feature + "]" + "still found, check removal code. (Should not see this)" )
        
df_five = df[['sttl','ct_dst_sport_ltm', 'ct_src_dport_ltm', 'swin', 'dwin', 'label' ]] 

duplicate record deleted successfully: 82328 observations remaining


* Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

In [3]:
df.describe()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
count,82328.000000,82328.000000,82328.000000,8.232800e+04,8.232800e+04,8.232800e+04,82328.000000,82328.00000,8.232800e+04,8.232800e+04,...,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000
mean,1.006783,18.666893,17.546303,7.994267e+03,1.323440e+04,8.241489e+04,180.973448,95.70541,6.455215e+07,6.305771e+05,...,4.929040,3.663092,7.456528,0.008187,0.008284,0.129749,6.468480,9.164610,0.011126,0.550578
std,4.710557,133.919593,115.576881,1.716464e+05,1.514751e+05,1.486229e+05,101.512436,116.66547,1.798656e+08,2.393055e+06,...,8.389724,5.915518,11.415443,0.090110,0.091439,0.638697,8.544117,11.121571,0.104893,0.497438
min,0.000000,1.000000,0.000000,2.400000e+01,0.000000e+00,0.000000e+00,0.000000,0.00000,0.000000e+00,0.000000e+00,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000008,2.000000,0.000000,1.140000e+02,0.000000e+00,2.860585e+01,62.000000,0.00000,1.120356e+04,0.000000e+00,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000
50%,0.014120,6.000000,2.000000,5.340000e+02,1.780000e+02,2.651198e+03,254.000000,29.00000,5.770751e+05,2.112632e+03,...,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,3.000000,5.000000,0.000000,1.000000
75%,0.719362,12.000000,10.000000,1.280000e+03,9.560000e+02,1.111111e+05,254.000000,252.00000,6.514286e+07,1.585818e+04,...,4.000000,3.000000,6.000000,0.000000,0.000000,0.000000,7.000000,11.000000,0.000000,1.000000
max,59.999989,10646.000000,11018.000000,1.435577e+07,1.465753e+07,1.000000e+06,255.000000,253.00000,5.268000e+09,2.082111e+07,...,59.000000,38.000000,63.000000,1.000000,2.000000,16.000000,60.000000,62.000000,1.000000,1.000000


In [14]:
# http://stackoverflow.com/questions/19482970/get-list-from-pandas-dataframe-column-headers

# Surrounding code in try/except on case where there are no object type features to one-hot encode
try:
    tmp_df = df.describe(include=['O'])  # creates a temporary df with just categorical features that are of object type
    categorical_object_col_name_list = tmp_df.columns.values.tolist()
    for col_name in categorical_object_col_name_list:
        #print col_name
        tmp_df = pd.get_dummies(df[col_name], prefix=col_name)
        df = pd.concat((df,tmp_df), axis=1)
        df.drop(col_name, axis=1, inplace=True)  # go ahead and drop original feature as it has now been one-hot encoded
except ValueError as e:
    print ("Value error({0}): ".format(e) ) # Note

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82328 entries, 0 to 82331
Columns: 191 entries, dur to state_RST
dtypes: float64(11), int64(29), uint8(151)
memory usage: 37.6 MB


In [6]:
df.head(10)

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_FIN,state_INT,state_REQ,state_RST
0,0.000011,2,0,496,0,90909.09020,254,0,1.803636e+08,0.0,...,0,0,0,0,0,0,0,1,0,0
1,0.000008,2,0,1762,0,125000.00030,254,0,8.810000e+08,0.0,...,0,0,0,0,0,0,0,1,0,0
2,0.000005,2,0,1068,0,200000.00510,254,0,8.544000e+08,0.0,...,0,0,0,0,0,0,0,1,0,0
3,0.000006,2,0,900,0,166666.66080,254,0,6.000000e+08,0.0,...,0,0,0,0,0,0,0,1,0,0
4,0.000010,2,0,2126,0,100000.00250,254,0,8.504000e+08,0.0,...,0,0,0,0,0,0,0,1,0,0
5,0.000003,2,0,784,0,333333.32150,254,0,1.045333e+09,0.0,...,0,0,0,0,0,0,0,1,0,0
6,0.000006,2,0,1960,0,166666.66080,254,0,1.306667e+09,0.0,...,0,0,0,0,0,0,0,1,0,0
7,0.000028,2,0,1384,0,35714.28522,254,0,1.977143e+08,0.0,...,0,0,0,0,0,0,0,1,0,0
8,0.000000,1,0,46,0,0.00000,0,0,0.000000e+00,0.0,...,0,0,0,0,0,0,0,1,0,0
9,0.000000,1,0,46,0,0.00000,0,0,0.000000e+00,0.0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
# we want to predict the X and y data as follows for 5 feature iteration: 
if 'label' in df_five:
    y = df_five['label'].values # get the labels we want
    del df_five['label'] # get rid of the class label
    X = df_five.values # use everything else to predict!

    # X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    # have converted them into simple matrices to use with scikit learn
    
    
# to use the cross validation object in scikit learn, we need to grab an instance
# of the object and set it up. This object will be able to split our data into 
# training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n=num_instances,
                         n_iter=num_cv_iterations,
                         test_size  = 0.2)
                         
print (cv_object)

ShuffleSplit(82328, n_iter=3, test_size=0.2, random_state=None)


In [21]:
dfcopy = df.copy(deep=True) # preserve original dataframe that has our dependent variable
dfcopy.info()
# we want to predict the X and y data as follows:
if 'label' in dfcopy:
    y = dfcopy['label'].values # get the labels we want
    del dfcopy['label'] # get rid of the class label
    X = dfcopy.values # use everything else to predict!

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82328 entries, 0 to 82331
Columns: 191 entries, dur to state_RST
dtypes: float64(11), int64(29), uint8(151)
memory usage: 37.6 MB


## Modeling and Evaluation:
* Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

*  Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.

*  Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms! 

In [37]:
numObservations = len(dfcopy)
print ("Number of observations= " + str(numObservations))

num_folds = 10
kf = KFold(numObservations, n_folds=num_folds)
skf = StratifiedKFold(y, num_folds)

num_trees = 100
max_features = 5

rf_clf = RandomForestClassifier(n_estimators=num_trees, max_features=max_features, n_jobs=-1)

t_total=0
for train_indices, test_indices in cv_object:
    t0 = time.clock()
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    print ('Training...')
    rf_clf.fit( X_train,y_train )

    print ('Predicting...')
    rf_clf.predict(X_test).astype(int)
    t1 = time.clock()
    t_total += t1 - t0

print ('Total training and predicition time was % .2f' % t_total + ' seconds.')

print ('\nEvaluating accuracies by cross validation...')
t0 = time.clock()
accuracies = cross_val_score(rf_clf, X, y=y, cv=skf)
t1 = time.clock()
accuracies_time = t1 - t0

print ('Evaluating precision by cross validation...')
t0 = time.clock()
precision = cross_val_score(rf_clf, X, y=y, cv=skf, scoring='precision')
t1 = time.clock()
precision_time = t1 - t0

print ('Evaluating recall by cross validation...')
t0 = time.clock()
recall = cross_val_score(rf_clf, X, y=y, cv=skf, scoring='recall')
t1 = time.clock()
recall_time = t1 - t0

print ('Evaluating accuracy by cross validation...')
t0 = time.clock()
accuracy = cross_val_score(rf_clf, X, y=y, cv=skf, scoring='accuracy')
t1 = time.clock()
accuracy_time = t1 - t0

print ('Evaluating f1 by cross validation...')
t0 = time.clock()
f1 = cross_val_score(rf_clf, X, y=y, cv=skf, scoring='f1')
t1 = time.clock()
f1_time = t1 - t0

print ("\nAverage Accuracies across stratified " + str(num_folds) + " fold cross validation iterations = {0:.2%} ".format(np.average(accuracies)) + ' accuracies cv time % .2f' % accuracies_time + ' seconds.')
print ("Average Precision across stratified " + str(num_folds) + " fold cross validation iterations = {0:.2%}".format(np.average(precision)) + ' precision cv time % .2f' % precision_time + ' seconds.')
print ("Average Recall(Sensitivity) across stratified " + str(num_folds) + " fold cross validation iterations = {0:.2%}".format(np.average(recall)) + ' recall cv time % .2f' % recall_time + ' seconds.')
print ("Average Accuracy across stratified " + str(num_folds) + " fold cross validation iterations = {0:.2%}".format(np.average(accuracy)) + ' accuracy cv time % .2f' % + accuracy_time + ' seconds.')
print ("Average F measure(F1) across stratified " + str(num_folds) + " fold cross validation iterations = {0:.2%}".format(np.average(f1)) + ' f1 cv time % .2f' % f1_time + ' seconds.')


Number of observations= 82328
Training...
Predicting...
Training...
Predicting...
Training...
Predicting...
Total training and predicition time was  21.60 seconds.

Evaluating accuracies by cross validation...
Evaluating precision by cross validation...
Evaluating recall by cross validation...
Evaluating accuracy by cross validation...
Evaluating f1 by cross validation...

Average Accuracies across stratified 10 fold cross validation iterations = 95.66%  accuracies cv time  62.58 seconds.
Average Precision across stratified 10 fold cross validation iterations = 96.85% precision cv time  62.04 seconds.
Average Recall(Sensitivity) across stratified 10 fold cross validation iterations = 95.25% recall cv time  63.16 seconds.
Average Accuracy across stratified 10 fold cross validation iterations = 95.59% accuracy cv time  62.47 seconds.
Average F measure(F1) across stratified 10 fold cross validation iterations = 95.97% f1 cv time  63.60 seconds.


*  Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

* Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation. 

https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms

### Logistic Regression ###
Logistic Regression (LR) is a fairly well-behaved classification algorithm which can be trained on features that are generally linear and as a rule of thumb is recommended as a starting point for classification to set a baseline for other classification comparrsions.  Additional advantages and disadvantages are:

**Advantages**
* Robust to noise
* Avoid overfitting and even do feature selection by using l2 or l1 regularization.
* Used in Big Data scenarios since it is pretty efficient
* Distributed using, for example, ADMM (see logreg).
* Output can be interpreted as a probability which allows for ranking instead of classification.
* Low variance
* Provides probabilities for outcomes
* Works well with diagonal (feature) decision boundaries
* NOTE: logistic regression can also be used with kernel methods

**Disadvantages**
* High bias
* Cannot deal with missing values
* Requiring to impute the missing values (or substitute them with a mean or median).


https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=5&cad=rja&uact=8&ved=0ahUKEwiJi87y243QAhXmjFQKHbEbB-cQFggvMAQ&url=http%3A%2F%2Fhome.etf.rs%2F~vm%2Fos%2Fdmsw%2FRandom%2520Forest.pptx&usg=AFQjCNEVo5hQOuo-6p2g3Tsa_snZfjlNnA&sig2=x5GDssT9R0WG1GaFb0pXYw&bvm=bv.137901846,d.cGw

### Random Forest ###
Random Forest (RF) is a tree ensemble that has several advantages over LR.  One of the main advantages of RF is linearity of features is not expected.  Tree ensembles such as RF handle categorical features well as well as high dimensional spaces and large number of training examples. Additional advantages and disadvantages are:

**Advantages**
* One of the more accurate learning algorithms available.
* Produces a highly accurate classifier for most datasets.
* Efficient on large databases.
* Supports thousands of input variables without variable deletion.
* Estimates what variables are important in the classification.
* It has an effective method for estimating missing data and maintains accuracy when a large proportion of the data are missing.
* Supports methods for balancing error in class population unbalanced data sets. 

**Disadvantages**
* Can overfit some datasets with noisy classification/regression tasks.
* Biased towards categorical variables with attributes containing more levels.  This can cause the variable importance scrores not to be reliable.


https://en.wikibooks.org/wiki/Data_Mining_Algorithms_In_R/Classification/kNN

### KNN ###
K-Nearest Neighbors (KNN) is a simple algorithm that stores all available cases and classifies new cases based on distance functions.  KNN can be used for both classification and regression predictive problems.

**Advantages**
* Very simple implementation.
* Robust with regard to the search space; for instance, classes don't have to be linearly separable.
* Classifier can be updated online at very little cost as new instances with known classes are presented.
* Few parameters to tune: distance and k.
* Offers noise reduction techniques that can be effective in improving the accuracy of the classifier.

**Disadvantages**
* Large storage requirements
* Computationally intensive recall
* Can have poor run-time performance if the training set is large
* Highly susceptible to dimensionality
* Expensive testing of each instance due to computing distance to all known instances. 
* Sensitive to noisy or irrelevant attributes, which can result in less meaningful distance numbers. 
* Sensitive to very unbalanced datasets, where most entities belong to one or a few classes.


* Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

## Deployment:
* How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

## Exceptional Work:

In [ ]:
df_copy = df.select_dtypes(['float64', 'int64'])

y = df.attack_cat


#######################################################
# Percentage of variance explained by first component
#######################################################
pca = PCA(n_components=1)
x_pca = pca.fit(df_copy).transform(df_copy)

# Percentage of variance explained for each component
print('explained variance ratio (first component): %3.2f'
      % (100 * pca.explained_variance_ratio_))


############################################################
# Percentage of variance explained for first two components
############################################################
pca = PCA(n_components=2)
x_pca = pca.fit(df_copy).transform(df_copy)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % (100 * pca.explained_variance_ratio_) )

 
##############################################################
# Percentage of variance explained for first three components
##############################################################
pca = PCA(n_components=3)
x_pca = pca.fit(df_copy).transform(df_copy) 

# Percentage of variance explained for each components
print('explained variance ratio (first three components): %s'
      % (100 * pca.explained_variance_ratio_) )

